In [1]:
 from theano import *

WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


In [3]:
#Importing Tensor subpackage
import theano.tensor as T
from theano import pp
from theano import function

## Basic Algebra

In [4]:
# x is a instance of TensorVariable of type dscalar
x = T.dscalar('x')
y = T.dscalar('y')

In [5]:
type(x)

theano.tensor.var.TensorVariable

In [6]:
x.type

TensorType(float64, scalar)

In [7]:
z = x+y
type(z)

theano.tensor.var.TensorVariable

In [10]:
# We can see compution associated with z
print pp(z)

(x + y)


In [11]:
# create a function taking x and y as inputs and giving z as output
f = function([x,y],z) # This will take some time as it will be compiled in c

In [12]:
f(2,3)

array(5.0)

We can just use variable eval method instead of function in this case. The eval() method is not as flexible as function().

In [14]:
# We passed eval() a dictionary mapping symbolic theano variables
# to the values to substitute for them
z.eval({x: 2, y: 3})

array(5.0)

### Adding two Matrices

In [15]:
x = T.dmatrix('x')
y = T.dmatrix('y')
z = x + y
f = function([x,y], z)

In [16]:
x = [[1,2], [3,4]]
y = [[-1,-2], [-3,-4]]
f(x,y)

array([[ 0.,  0.],
       [ 0.,  0.]])

### Simple Exercises

#### Example 1

In [19]:
# Get a + a^10 
i = T.vector('a')
out = i + i**10
f = function([i],out)
f([0,1,2])

array([    0.,     2.,  1026.], dtype=float32)

#### Example 2

In [22]:
# Get a^2 + b^2 + 2ab
a = T.vector('a')
b = T.vector('b')
out = a**2 + b**2 + 2*a*b
f = function([a,b], out)
f([1,3], [2,5])

array([  9.,  64.], dtype=float32)

### More Examples

#### Logistic Function

In [4]:
x = T.dmatrix('x')
s = 1 / (1 + T.exp(-x))
logistic = function([x],s)
logistic([[10,1], [0,-50]])

array([[ 0.9999546 ,  0.73105858],
       [ 0.5       ,  0.        ]])

#### Computing More than one Thing at the Same Time
Theano supports functions with multiple outputs

In [9]:
a, b = T.dmatrices('a', 'b')
diff = a-b
abs_diff = abs(diff)
diff_sqr = diff**2
f = function([a,b], [diff, abs_diff, diff_sqr])
f([[2]], [[3]])

[array([[-1.]]), array([[ 1.]]), array([[ 1.]])]

#### Using Shared Variables
It is also possible to make a function with an internal state. For example, let’s say we want to make an accumulator: at the beginning, the state is initialized to zero. Then, on each function call, the state is incremented by the function’s argument.

In [13]:
from theano import shared
state = shared(0)
inc = T.iscalar('inc')
accumulator = function([inc], state, updates = [(state, state+inc)])

In [14]:
print(state.get_value())
accumulator(5)
print(state.get_value())
accumulator(15)
print (state.get_value())
accumulator(300)
print(state.get_value())

0
5
20
320


#### Copying functions
Theano functions can be copied, which can be useful for creating similar functions but with different shared variables or updates. This is done using the copy() method of function objects. The optimized graph of the original function is copied, so compilation only needs to be performed once.

In [15]:
new_state = shared(0)
new_acc = accumulator.copy(swap={state:new_state})

#### Random Numbers

In [17]:
from theano.tensor.shared_randomstreams import RandomStreams